# Convolutional Variational Autoencoder (CVAE) <br> For Time Series Augmentaion

<br>

## Input the user-defined parameters 

In [ ]:
# File directory (change the directory to the folder including your time series data)
file_dir = 'C:/Users/jiayi/Desktop/CVAE Codes for GitHub/'


# Latent space size (usually it is a small number e.g., <= 8)
latent_dim = 4


# Number of epochs
NUM_EPOCH = 50000


# Stop training when loss reaching
Loss_Stop = 0.0000


# Initial learning rate for Adam optimizer
Learning_Rate = 0.005


# Initial learning rate step decay parameters
Drop_Rate1 = 0.80
NUM_EPOCH_DROP1 = 5000

Drop_Rate2 = 0.90
NUM_EPOCH_DROP2 = 10000

Drop_Rate3 = 0.95
NUM_EPOCH_DROP3 = 20000

Drop_Rate4 = 1.00


# Drop learning rate every Drop_Epoch
Drop_Epoch = 500


# Define a GPU usage
# CPU = -1, GPU0 = 0, GPU1 = 1
GPU_ID = -1

<br>

## Load packages and data

In [ ]:
# Check if the GPU is specified
import os
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID) 

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

In [ ]:
# Create folders for model, data, and figure
import shutil

# Folder name
ID = f"_Lat{latent_dim}_Epoch{NUM_EPOCH}" 

# Model
model_dir = f"Model{str(ID)}" 
modelExist = os.path.exists(model_dir)
if modelExist:
    shutil.rmtree(model_dir)
    os.makedirs(model_dir) 
    print('Model folder exists, delete the current files in this folder.')
else:
    os.makedirs(model_dir)    
    print('Model folder does not exist, create this folder.')
    
# Data
data_dir = f"Data{str(ID)}" 
dataExist = os.path.exists(data_dir)
if dataExist:
    shutil.rmtree(data_dir)
    os.makedirs(data_dir) 
    print('Data folder exists, delete the current files in this folder.')
else:
    os.makedirs(data_dir)    
    print('Data folder does not exist, create this folder.')
        
# Figure
figure_dir = f"Figure{str(ID)}" 
figureExist = os.path.exists(figure_dir)
if figureExist:
    shutil.rmtree(figure_dir)    
    os.makedirs(figure_dir) 
    print('Figure folder exists, delete the current files in this folder.')
else:
    os.makedirs(figure_dir)    
    print('Figure folder does not exist, create this folder.')

In [ ]:
# Load packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Conv2DTranspose, Lambda, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras import initializers
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
import scipy.io
from tensorflow import keras
from tensorflow.keras.utils import plot_model
import time
from keras.layers import ReLU
from keras.layers import PReLU
from keras.layers import LeakyReLU

In [ ]:
# Load the time series data
Data = scipy.io.loadmat(file_dir + 'TimeSeries.mat')
# Notes: 
# 'TimeSeries.mat' is a Nt*1 vector of cells (Nt is the number of time series).
# Each cell includes a Ns*D matrix (Ni is the number of time steps, D is the number of dimensions).

T_Raw = Data["TimeSeries"]     

# Reshape the time series data
T_Raw = np.squeeze(T_Raw)
T = np.empty((T_Raw.shape[0], T_Raw[0].shape[0], T_Raw[0].shape[1]))
for i in range(T_Raw.shape[0]):
    T[i] = T_Raw[i]

T = T.reshape(-1, T.shape[1],T.shape[2], 1)

print('')
print('Time series data shape = ', T.shape)
print('')
print('Number of time series =', T.shape[0])
print('Number of time steps in each time series =', T.shape[1])
print('Number of dimensions in each time series = ', T.shape[2])

<br>

## Encoder
### Note: The activation function, number of filters, kernel size, pooling size, strides, padding, and number of nodes in dense layers can be defined based on user's preference. 

In [ ]:
# Input layer
input_encoder = Input(shape=(T.shape[1],T.shape[2],T.shape[3]), name = 'x_T')


# Activation function
act_fun = LeakyReLU(alpha=0.5)
# act_fun = 'tanh'
# act_fun = 'ReLU'
# act_fun = 'PReLU'


# Convolutional and max pooling layers
x_C1 = Conv2D(filters = 16, kernel_size = (3,3), padding="same",
              activation=act_fun, name = 'x_C1',
              kernel_initializer=initializers.GlorotNormal,
              bias_initializer=initializers.Zeros())(input_encoder)

x_P1 = MaxPooling2D(pool_size = (3,1), strides = (2,1), padding='same', name = 'x_P1')(x_C1)

x_C2 = Conv2D(filters = 8, kernel_size = (3,3), padding="same", 
              activation=act_fun, name = 'x_C2',
              kernel_initializer=initializers.GlorotNormal,
              bias_initializer=initializers.Zeros())(x_P1)

x_P2 = MaxPooling2D(pool_size = (3,1), strides = (2,1), padding='same', name = 'x_P2')(x_C2)

x_C3 = Conv2D(filters = 4, kernel_size = (3,3), padding="same",
              activation=act_fun, name = 'x_C3',
              kernel_initializer=initializers.GlorotNormal,
              bias_initializer=initializers.Zeros())(x_P2)


# Flatten layer
x_F = Flatten(name = 'x_F')(x_C3)


# Dense layers
x_D1 = Dense(512, activation=act_fun, name = 'x_D1',
                  kernel_initializer=initializers.GlorotNormal,
                  bias_initializer=initializers.Zeros())(x_F)
x_D2 = Dense(128, activation=act_fun, name = 'x_D2',
                  kernel_initializer=initializers.GlorotNormal,
                  bias_initializer=initializers.Zeros())(x_D1)
x_D3 = Dense(32, activation=act_fun, name = 'x_D3',
                  kernel_initializer=initializers.GlorotNormal,
                  bias_initializer=initializers.Zeros())(x_D2)
x_D4 = Dense(8, activation=act_fun, name = 'x_D4',
                 kernel_initializer=initializers.GlorotNormal,
                 bias_initializer=initializers.Zeros())(x_D3)


# Latent feature layer
# Mean
encoder_mu = Dense(latent_dim, name="Encoder_Mu")(x_D4)
# Log-variance
encoder_log_variance = Dense(latent_dim, name="Encoder_LogVariance")(x_D4)


# Sampling
def sampling(mu_log_variance):
    mu, log_variance = mu_log_variance
    epsilon = tensorflow.keras.backend.random_normal(shape=tensorflow.keras.backend.shape(mu), mean=0.0, stddev=1.0)
    random_sample = mu + tensorflow.keras.backend.exp(log_variance)*epsilon
    return random_sample


# Encoder
output_encoder = Lambda(sampling, name="Encoder_Output")([encoder_mu, encoder_log_variance])
encoder = Model([input_encoder], [output_encoder], name='encoder')

In [ ]:
# View the encoder information
encoder.summary()

In [ ]:
# Plot the encoder architecture
plot_model(encoder, figure_dir + "/Encoder.png", show_shapes=True)

</br>

## Decoder
### Note: The number of filters, kernel size, strides, padding, and number of nodes in dense layers are symmetric to the ones for encoder. 

In [ ]:
# Input layer
input_decoder = Input(shape=(latent_dim), name = 'Decoder_Input')


# Denses layer
y_D1 = Dense(8, activation=act_fun, name = 'y_D1',
                kernel_initializer=initializers.GlorotNormal,
                bias_initializer=initializers.Zeros())(input_decoder)
y_D2 = Dense(32, activation=act_fun, name = 'y_D2',
                 kernel_initializer=initializers.GlorotNormal,
                 bias_initializer=initializers.Zeros())(y_D1)
y_D3 = Dense(128, activation=act_fun, name = 'y_D3',
                  kernel_initializer=initializers.GlorotNormal,
                  bias_initializer=initializers.Zeros())(y_D2)
y_D4 = Dense(512, activation=act_fun, name = 'y_D4',
                  kernel_initializer=initializers.GlorotNormal,
                  bias_initializer=initializers.Zeros())(y_D3)
y_D5 = Dense(x_F.shape[1], activation=act_fun, name = 'y_D5',
                           kernel_initializer=initializers.GlorotNormal,
                           bias_initializer=initializers.Zeros())(y_D4)


# Inverse flatten layer
y_F = Reshape([x_C3.shape[1], x_C3.shape[2],x_C3.shape[3]], name = 'y_F')(y_D5)


# Deconvolutional layers
y_C1 = Conv2DTranspose(filters = 8, strides = (1,1), kernel_size = (3,3), padding="same", 
                       activation=act_fun, name = 'y_C1',
                       kernel_initializer=initializers.GlorotNormal,
                       bias_initializer=initializers.Zeros())(y_F)                                                     

y_C2 = Conv2DTranspose(filters = 8, strides = (2,1), kernel_size = (3,3), padding="same", 
                       activation=act_fun, name = 'y_C2',
                       kernel_initializer=initializers.GlorotNormal,
                       bias_initializer=initializers.Zeros())(y_C1)                                                     

y_C3 = Conv2DTranspose(filters = 16, strides = (1,1), kernel_size = (3,3), padding="same", 
                       activation=act_fun, name = 'y_C3',
                       kernel_initializer=initializers.GlorotNormal,
                       bias_initializer=initializers.Zeros())(y_C2)  

y_C4 = Conv2DTranspose(filters = 16, strides = (2,1), kernel_size = (3,3), padding="same", 
                       activation=act_fun, name = 'y_C4',
                       kernel_initializer=initializers.GlorotNormal,
                       bias_initializer=initializers.Zeros())(y_C3)   

y_C5 = Conv2DTranspose(filters = 1, kernel_size = (3,3), padding="same", 
                       activation=act_fun, name = 'y_C5',
                       kernel_initializer=initializers.GlorotNormal,
                       bias_initializer=initializers.Zeros())(y_C4)    

output_decoder = Conv2DTranspose(filters = 1, kernel_size = (3,3), padding="same", 
                                 activation='linear', name = 'Decorder_Output',
                                 kernel_initializer=initializers.GlorotNormal,
                                 bias_initializer=initializers.Zeros())(y_C5) 


# Decoder
decoder = Model([input_decoder], [output_decoder], name='decoder')

In [ ]:
# View the decoder information
decoder.summary()

In [ ]:
# Plot the decoder architecture
plot_model(decoder, figure_dir + "/Decoder.png", show_shapes=True)

<br>

## Autoencoder

In [ ]:
# Input
input_auto = input_encoder

# Output
output_auto = decoder(encoder(input_encoder))

# Autoencoder
autoencoder = Model([input_auto], [output_auto], name='auto')

In [ ]:
# View the autoencoder information
autoencoder.summary()

In [ ]:
# Plot the autoencoder architecture
plot_model(autoencoder, figure_dir + "/Autoencoder.png", show_shapes=True)

<br>

## Training parameters

In [ ]:
# Loss Function
# Mean squared error
mse = tf.keras.losses.MeanSquaredError()
loss_mse = mse(input_auto, output_auto)

# KL-divergence
loss_kl = -0.5*tf.keras.backend.sum(1.0 + encoder_log_variance - tf.keras.backend.square(encoder_mu) - tensorflow.keras.backend.exp(encoder_log_variance), axis=1)
   
# Add loss to autoencoder 
loss = loss_mse + loss_kl*0.001
autoencoder.add_loss(loss) 

In [ ]:
# Adam optimizer
autoencoder.compile(optimizer=optimizers.Adam(learning_rate=Learning_Rate))

# Initial learning rate step decay 
def lr_step_decay(epoch, lr):
    if epoch < NUM_EPOCH_DROP1:
        return Learning_Rate * math.pow(Drop_Rate1, math.floor(epoch/Drop_Epoch))
    if ((epoch >= NUM_EPOCH_DROP1) and (epoch < NUM_EPOCH_DROP2)):
        return Learning_Rate * math.pow(Drop_Rate1, math.floor(NUM_EPOCH_DROP1/Drop_Epoch)) * math.pow(Drop_Rate2, math.floor((epoch-NUM_EPOCH_DROP1)/Drop_Epoch))
    if ((epoch >= NUM_EPOCH_DROP2) and (epoch < NUM_EPOCH_DROP3)):
        return Learning_Rate * math.pow(Drop_Rate1, math.floor(NUM_EPOCH_DROP1/Drop_Epoch)) * math.pow(Drop_Rate2, math.floor((NUM_EPOCH_DROP2-NUM_EPOCH_DROP1)/Drop_Epoch)) * math.pow(Drop_Rate3, math.floor((epoch-NUM_EPOCH_DROP2)/Drop_Epoch))
    if epoch >= NUM_EPOCH_DROP3:
        return Learning_Rate * math.pow(Drop_Rate1, math.floor(NUM_EPOCH_DROP1/Drop_Epoch)) * math.pow(Drop_Rate2, math.floor((NUM_EPOCH_DROP2-NUM_EPOCH_DROP1)/Drop_Epoch)) * math.pow(Drop_Rate3, math.floor((NUM_EPOCH_DROP3-NUM_EPOCH_DROP2)/Drop_Epoch)) * math.pow(Drop_Rate4, math.floor((epoch-NUM_EPOCH_DROP3)/Drop_Epoch))
              
LR_Scheduler = tf.keras.callbacks.LearningRateScheduler(lr_step_decay, verbose=0)

In [ ]:
# Checkpoint path and direction
checkpoint_path = model_dir + "/Checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# Callback to save the model's weights at the best epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 verbose = 1, 
                                                 monitor='loss',
                                                 mode='min',
                                                 save_weights_only = True,
                                                 save_best_only = True)

In [ ]:
# Early stop when the loss reaches a user-defined value
class haltCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss') <= Loss_Stop):
            print("\n\n\nReached %s loss value, so stopping training!\n\n\n"  %(str(Loss_Stop)))
            self.model.stop_training = True

trainingStopCallback = haltCallback()

<br>

## Train the autoencoder

In [ ]:
# Input and output data
Input_set = T
Output_set = T

# Batch size
BATCH_SIZE = T.shape[0]

# Training
start = time.time()
history = autoencoder.fit(Input_set, Output_set,
                          epochs = NUM_EPOCH,
                          batch_size = BATCH_SIZE,
                          callbacks = [cp_callback, trainingStopCallback, LR_Scheduler],
                          verbose = 0)
end = time.time()

In [ ]:
# Print and save the running time
print("The running time:", round((end-start)/60,1), "mins")
print('')

Time = np.asarray(round((end-start)/60,1))
Time = Time.reshape(1,-1)
np.savetxt(data_dir + "/Running_Time.txt", Time);

In [ ]:
# Find the best checkpoint, which is associated with the smallest loss value
loss_min_index = history.history['loss']. index(min(history.history['loss']))
ckpt_best = loss_min_index+1
print('Number of epochs at the minimum loss = ', ckpt_best)
print('')

Min_Loss_Epoch = open(data_dir + '/Min_Loss_Epoch.dat', 'w')
Min_Loss_Epoch.write(str(ckpt_best))
Min_Loss_Epoch.close()

In [ ]:
# Checkpoint ID
Checkpoint_ID = os.path.join(model_dir, "Checkpoint.ckpt")

<br>

## Save results

In [ ]:
# Autoencoder
autoencoder_best = autoencoder
autoencoder_best.load_weights(Checkpoint_ID)

# Autoencoder output
auto_output_raw = autoencoder_best.predict(T,verbose = 0);
auto_output = auto_output_raw.reshape(T.shape[0],T.shape[1]*T.shape[2])
np.savetxt(data_dir + '/Autoencoder_Output.dat', auto_output);

In [ ]:
# Encoder
encoder_best = encoder

# Load weights from autoencoder_best
encoder_len = len(encoder_best.weights)
encoder_best.set_weights(autoencoder_best.weights[0:encoder_len])

# mu
layer_mu = 'Encoder_Mu'
mu_best = Model(inputs = autoencoder_best.input, outputs = autoencoder_best.get_layer(layer_mu).output);
mu = mu_best.predict(T,verbose = 0);
np.savetxt(data_dir + '/Encode_Mu.dat', mu);

# log variance
layer_logvariance = 'Encoder_LogVariance'
logvariance_best = Model(inputs = autoencoder_best.input, outputs = autoencoder_best.get_layer(layer_logvariance).output);
logvariance = logvariance_best.predict(T,verbose = 0);
np.savetxt(data_dir + '/Encode_LogVariance.dat', logvariance);

# Encoder output
layer_encoder_output = 'encoder'
encoder_output_best = Model(inputs = autoencoder_best.input, outputs = autoencoder_best.get_layer(layer_encoder_output).output);
encoder_output = encoder_output_best.predict(T,verbose = 0);
np.savetxt(data_dir + '/Encode_Output.dat', encoder_output);

In [ ]:
# Decoder
decoder_best = decoder

# Load weights from autoencoder_best
decoder_len = len(decoder_best.weights)
decoder_best.set_weights(autoencoder_best.weights[encoder_len:encoder_len+decoder_len])

# Decoder output
decoder_output_raw = decoder_best.predict(encoder_output,verbose = 0);
decoder_output = decoder_output_raw.reshape(T.shape[0],T.shape[1]*T.shape[2])
np.savetxt(data_dir + '/Decoder_Output.dat', decoder_output);

In [ ]:
# loss
np.savetxt(data_dir + '/Loss.dat', history.history['loss'])